In [139]:
from bs4 import BeautifulSoup
import requests
import urllib.request as req
import pandas as pd
import folium

# 1단계 : 서울 문화공간 위치정보(주소) 획득
key = '4a6353746b796a68333079486b464e' # 7549474863796a68383358725a7975
url1 = 'http://openAPI.seoul.go.kr:8088/' + key + '/xml/SearchCulturalFacilitiesAddressService/1/1000'
save_attrfile_path = 'C:/Users/Performance/Desktop/Fist/src/ch15_attr_xmlfile.xml'
req.urlretrieve(url1, save_attrfile_path)

attr_xml = open(save_attrfile_path, 'r', encoding='utf-8').read() 
soup = BeautifulSoup(attr_xml, 'xml')

attr_list = []
for row in soup.select('row'):
    fac_code = row.select_one('FAC_CODE').text
    subjcode = row.select_one('SUBJCODE').text
    codename = row.select_one('CODENAME').text
    fac_name = row.select_one('FAC_NAME').text
#     main_img = row.select_one('MAIN_IMG') ######AttributeError: 'NoneType' object has no attribute 'text'##########
    addr = row.select_one('ADDR').text.replace(' )','')
#     attr_list.append( {'fac_code' : fac_code, 'subj_code' : subjcode, 'codename' : codename, 'fac_name' : fac_name, 'main_img' : main_img, 'addr' : addr} )
    attr_list.append( {'fac_code' : fac_code, 'subj_code' : subjcode, 'codename' : codename, 'fac_name' : fac_name, 'addr' : addr} )
df_attr = pd.DataFrame(attr_list)



# 2단계 : 위치정보(주소)를 좌표로 변환
url2 = 'http://api.vworld.kr/req/address?service=address&request=getCoord&version=2.0&crs=epsg:4326&format=json&key=EA773332-297D-38CA-9A54-76F66D21A455&type=parcel'
addr_list = []
for idx, addr in enumerate(df_attr['addr']):
    try:
        resp = requests.get(url2 + '&address=' + addr)
        addr_json = resp.json()
        if addr_json['response']['status'] == 'OK':
            dic = {}
            dic['level2'] = addr_json['response']['refined']['structure']['level2']
            dic['x'] = addr_json['response']['result']['point']['x']
            dic['y'] = addr_json['response']['result']['point']['y']
            addr_list.append(dic)
        elif addr_json['response']['status'] == 'NOT_FOUND':
            url2 = 'http://api.vworld.kr/req/address?service=address&request=getCoord&version=2.0&crs=epsg:4326&format=json&key=EA773332-297D-38CA-9A54-76F66D21A455&type=road'
            resp2 = requests.get(url2 + '&address=' + addr)
            addr_json = resp2.json()
            if addr_json['response']['status'] == 'OK':
                dic = {}
                dic['level2'] = addr_json['response']['refined']['structure']['level2']
                dic['x'] = addr_json['response']['result']['point']['x']
                dic['y'] = addr_json['response']['result']['point']['y']
                addr_list.append(dic)
            else:
                dic = {}
                dic['level2'] = None
                dic['x'] = None
                dic['y'] = None
                addr_list.append(dic)                
        else:
            dic = {}
            dic['level2'] = None
            dic['x'] = None
            dic['y'] = None
            addr_list.append(dic)
    except Exception:
        dic = {}
        dic['level2'] = None
        dic['x'] = None
        dic['y'] = None
        addr_list.append(dic)
df_addr = pd.DataFrame(addr_list)

if df_attr.iloc[:,0].size == df_addr.iloc[:,0].size:
    df_combine = pd.concat([df_attr, df_addr], axis = 1)
else:
    print('두 데이터프레임의 행 수가 일치하지 않습니다.')

def fn1_by_district():
    import webbrowser
    dist_list = []
    while True:
        dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)')
        if dist_input == '':
            break
        dist_list.append(dist_input)

    df_list = []
    for dist in dist_list:
        df_list.append(df_combine[df_combine['level2']==dist])    
    df_result1 = pd.concat(df_list)

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result1['y'].astype('float').mean()
    long_mean = df_result1['x'].astype('float').mean()
    m = folium.Map([lat_mean, long_mean], zoom_start = 14)

    result1_by_subj = []
    for idx, n in enumerate(df_result1['subj_code'].unique()): # enumerate를 생각해낸게 해결법에 가까워진듯.
        result1_by_subj.append(df_result1[df_result1['subj_code'] == n])    
        for k in result1_by_subj[idx].index:
            tooltip = df_result1.loc[k, 'codename'] + ' - ' + df_result1.loc[k, 'fac_name'] + ' - ' + df_result1.loc[k, 'addr']
            lat = df_result1.loc[k, 'y']
            long = df_result1.loc[k, 'x']   # result1 대신 result1_by_subj가 들어가야 하지 않을까 ?
            folium.CircleMarker(
                location = [lat, long],
                radius = 3,
                color = hex_color[idx],
                tooltip = tooltip
            ).add_to(m)

    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

def fn2_by_type():
    import webbrowser
    attr_type_list = []
    while True:
        type_input = input('[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요')
        if type_input == '':
            break
        attr_type_list.append(type_input)
    
    type_matched_list = []
    for attr_type in attr_type_list:
        type_matched_list.append(df_combine[df_combine['codename'] == attr_type])
    df_result2 = pd.concat(type_matched_list)
    df_result2.dropna(axis = 0, how = 'any', inplace = True)
    
    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result2['y'].astype('float').mean()
    long_mean = df_result2['x'].astype('float').mean()
    m = folium.Map([lat_mean, long_mean], zoom_start = 14)
    
    for idx, c in enumerate(attr_type_list):
        for ele in df_result2[df_result2['codename'] == c].index:
            tooltip = df_result2.loc[ele,'codename'] + ' - ' + df_result2.loc[ele, 'fac_name'] + ' - '+ df_result2.loc[ele, 'addr']
            lat = df_result2.loc[ele, 'y']
            long = df_result2.loc[ele, 'x']
            folium.CircleMarker(
                location = [lat, long],
                radius = 3,
                color = hex_color[idx],
                tooltip = tooltip
            ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

def fn3_by_multi():
    import webbrowser
    dist = input('희망하는 지역의 행정구역(구)를 입력해주세요')
    attr_type = input('[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요')
    df_result3 = df_combine[(df_combine['level2'] == dist) & (df_combine['codename'] == attr_type)]

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result3['y'].astype('float').mean()
    long_mean = df_result3['x'].astype('float').mean()
    
    m = folium.Map([lat_mean, long_mean], zoom_start=14)
    for idx, l in enumerate(df_result3['codename'].unique()): # 나중에 이중 for문이 필요 없음을 깨닫고 삭제함.
        for i in df_result3[df_result3['codename'] == l].index:
            lat = df_result3.loc[i, 'y']
            long = df_result3.loc[i, 'x']
            tooltip = df_result3.loc[i, 'codename'] + ' - ' + df_result3.loc[i, 'fac_name'] + ' - ' + df_result3.loc[k, 'addr']
            folium.CircleMarker(
                location = [lat, long],
                radius = 3,
                color = hex_color[idx],
                tooltip = tooltip,
            ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')


# def fn2_by_type():
#     type_list = []
#     while True:
#         type_input = input('공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요')
#         if type_input == '':
#             break
#     type_list.append(type_input)
    
def main():
    while True:
        print('1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료')
        fn = int(input('선택 : '))
        if fn == 1:
            fn1_by_district()
        elif fn == 2:
            fn2_by_type()
        elif fn == 3:
            fn3_by_multi()
        elif fn == 9:
            break

In [140]:
if __name__ == '__main__':
    main()

1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료
선택 : 1
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)용산구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)서대문구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)
1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료
선택 : 2
[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요공연장
[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요미술관
[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요
1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료
선택 : 3
희망하는 지역의 행정구역(구)를 입력해주세요성동구
[ 공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 ] 중 선택해주세요기타
1: 행정구역 내 관광지 | 2: 장르별 관광지 | 3: 행정구역 내 장르별 관광지 | 9: 종료
선택 : 9


In [2]:
df_combine # 위에 셀 완료되면 불러와 지는지 실행해 볼 것

,fac_code,subj_code,codename,fac_name,addr,level2,x,y
0,101124,1,공연장,신도림 오페라하우스,서울시 구로구 구로동 3-39,None,None,None
1,101304,1,공연장,신촌문화발전소,서울특별시 서대문구 연세로2나길 57,서대문구,126.939751957,37.557984740
2,101429,1,공연장,서울광장,서울특별시 중구 태평로2가 17-3,None,None,None
3,101193,1,공연장,오류아트홀,"서울특별시 구로구 경인로20가길 38, 오류문화센터 2층(오류아트홀)",구로구,126.846035842,37.495358095
4,101183,1,공연장,정효아트센터,서울특별시 서초구 사임당로18길 52-2,서초구,127.019623092,37.487534940
...,...,...,...,...,...,...,...,...
908,100229,11,기타,삼청각,서울 성북구 성북동 330-115,None,None,None
909,100506,11,기타,꼬스트홀,서울 중구 명동2가 1-1,None,None,None
910,100738,11,기타,중구청,서울 중구 예관동 120-1,None,None,None
911,100548,11,기타,남산골 한옥마을,서울 중구 필동2가 84-1,None,None,None


In [85]:
dist_list = []
while True:
    dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)')
    if dist_input == '':
        break
    dist_list.append(dist_input)

df_list = []
for dist in dist_list:
    df_list.append(df_combine[df_combine['level2']==dist])    
df_result1 = pd.concat(df_list)
df_result1.dropna(axis = 0, how = 'any', inplace = True)

hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
lat_mean = df_result1['y'].astype('float').mean()
long_mean = df_result1['x'].astype('float').mean()
m = folium.Map([lat_mean, long_mean], zoom_start = 14)

for idx, n in enumerate(df_result1['subj_code'].unique()): # enumerate를 생각해낸게 해결법에 가까워진듯. 
    for k in df_result1[df_result1['subj_code'] == n].index:
        tooltip = df_result1.loc[k, 'codename'] + ' - ' + df_result1.loc[k, 'fac_name'] + ' - ' + df_result1.loc[k, 'addr']
        lat = df_result1.loc[k, 'y']
        long = df_result1.loc[k, 'x']   # result1 대신 result1_by_subj[idx]가 들어가야 하지 않을까 ?
        folium.CircleMarker(
            location = [lat, long],
            radius = 3,
            color = hex_color[idx],
            tooltip = tooltip
        ).add_to(m)
m

희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)종로구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)중구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)서대문구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)


In [57]:
def fn1_by_district():
    import webbrowser
    dist_list = []
    while True:
        dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)')
        if dist_input == '':
            break
        dist_list.append(dist_input)

    df_list = []
    for dist in dist_list:
        df_list.append(df_combine[df_combine['level2']==dist])    
    df_result1 = pd.concat(df_list)
    df_result1.dropna(axis = 0, how = 'any', inplace = True) # dropna 안하고도 None값이 제외되네 ?

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result1['y'].astype('float').mean()
    long_mean = df_result1['x'].astype('float').mean()
    m = folium.Map([lat_mean, long_mean], zoom_start = 14)

    for idx, n in enumerate(df_result1['subj_code'].unique()): # enumerate를 생각해낸게 해결법에 가까워진듯. 
        for k in df_result1[df_result1['subj_code'] == n].index:
            tooltip = df_result1.loc[k, 'codename'] + ' - ' + df_result1.loc[k, 'fac_name'] + ' - ' + df_result1.loc[k, 'addr']
            lat = df_result1.loc[k, 'y']
            long = df_result1.loc[k, 'x']   # result1 대신 result1_by_subj[idx]가 들어가야 하지 않을까 ?
            folium.CircleMarker(
                location = [lat, long],
                radius = 3,
                color = hex_color[idx],
                tooltip = tooltip
            ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

In [30]:
m.save('ch15_project_portfolio.html')
webbrowser.open('ch15_project_portfolio.html')

import webbrowser
webbrowser.open('ch15_project_portfolio.html')

True

In [58]:
fn1_by_district()

희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)종로구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)중구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)서대문구
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)


In [98]:
attr_type_list = []
while True:
    type_input = input('공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요')
    if type_input == '':
        break
    attr_type_list.append(type_input)

type_matched_list = []
for attr_type in attr_type_list:
    type_matched_list.append(df_combine[df_combine['codename'] == attr_type])
df_result2 = pd.concat(type_matched_list)
df_result2.dropna(axis = 0, how = 'any', inplace = True)
df_result2.index

공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요기타
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요도서관
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요문화원
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요


Int64Index([696, 697, 699, 700, 702, 703, 704, 705, 708, 709,
            ...
            533, 534, 538, 540, 545, 547, 554, 555, 556, 561],
           dtype='int64', length=266)

In [104]:
print(len(df_result2.index))
print(df_result2.iloc[:,0].size)

266
266


In [115]:
df_result2.index

Int64Index([696, 697, 699, 700, 702, 703, 704, 705, 708, 709,
            ...
            533, 534, 538, 540, 545, 547, 554, 555, 556, 561],
           dtype='int64', length=266)

In [121]:
df_result2.columns

Index(['fac_code', 'subj_code', 'codename', 'fac_name', 'addr', 'level2', 'x',
       'y'],
      dtype='object')

In [122]:
def fn2_by_type():
    import webbrowser
#     global attr_type_list # 여기에 global을 선언하지 않으면 fn2 함수 실행결과 type_list가 존재하지 않는 것으로 나옴.
#             # NameError: name 'type_list' is not defined
    attr_type_list = []
    while True:
        type_input = input('공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요')
        if type_input == '':
            break
        attr_type_list.append(type_input)
    
    type_matched_list = []
    for attr_type in attr_type_list:
        type_matched_list.append(df_combine[df_combine['codename'] == attr_type])
    df_result2 = pd.concat(type_matched_list)
    df_result2.dropna(axis = 0, how = 'any', inplace = True)
    
    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result2['y'].astype('float').mean()
    long_mean = df_result2['x'].astype('float').mean()
    m = folium.Map([lat_mean, long_mean], zoom_start = 14)
    
    for idx, c in enumerate(attr_type_list):
        for ele in df_result2[df_result2['codename'] == c].index:
            tooltip = df_result2.loc[ele,'codename'] + ' - ' + df_result2.loc[ele, 'fac_name'] + ' - '+ df_result2.loc[ele, 'addr']
            lat = df_result2.loc[ele, 'y']
            long = df_result2.loc[ele, 'x']
            folium.CircleMarker(
                location = [lat, long],
                radius = 3,
                color = hex_color[idx],
                tooltip = tooltip
            ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

# Location should consist of two numerical values, but None of type <class 'NoneType'> is not convertible to float.
# 왜 fn1에서는 None값으로 인한 오류가 발생하지 않았었지?

In [123]:
fn2_by_type()

공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요기타
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요도서관
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요문화원
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요


In [ ]:
# fn3은 지역 + 종류를 입력받고 comtain함수로 찾아볼 수 있으려나 ?

In [134]:
def fn3_by_multi():
    import webbrowser
    dist = input('행정구역 명을 입력해주세요.')
    attr_type = input('공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요')
    df_result3 = df_combine[(df_combine['level2'] == dist) & (df_combine['codename'] == attr_type)]

    hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
    lat_mean = df_result3['y'].astype('float').mean()
    long_mean = df_result3['x'].astype('float').mean()
    
    m = folium.Map([lat_mean, long_mean], zoom_start=14)
    for idx, l in enumerate(df_result3['codename'].unique()):
        for i in df_result3[df_result3['codename'] == l].index:
            lat = df_result3.loc[i, 'y']
            long = df_result3.loc[i, 'x']
            tooltip = df_result3.loc[i, 'codename'] + ' - ' + df_result3.loc[i, 'fac_name'] + ' - ' + df_result1.loc[k, 'addr']
            folium.CircleMarker(
                location = [lat, long],
                radius = 3,
                color = hex_color[idx],
                tooltip = tooltip,
            ).add_to(m)
    m.save('ch15_project_portfolio.html')
    webbrowser.open('ch15_project_portfolio.html')

In [129]:
df_combine.columns

Index(['fac_code', 'subj_code', 'codename', 'fac_name', 'addr', 'level2', 'x',
       'y'],
      dtype='object')

In [135]:
fn3_by_multi()

행정구역 명을 입력해주세요.종로구
공연장 | 박물관/기념관 | 미술관 | 문화예술회관 | 문화원 | 도서관 | 기타 중 선택해주세요미술관


In [126]:
# 3단계 : 문화공간을 지도에 표시
dist_list = []
while True:
    dist_input = input('관광지위치 확인을 희망하는 행정구역(구)를 입력해주세요')
    if dist_input == '':
        break
    dist_list.append(dist_input)

df_list = []
for dist in dist_list:
    df_list.append(df_combine[df_combine['level2']==dist])    
df_result = pd.concat(df_list)
df_result.dropna(axis = 0, how = 'any', inplace = True)

hex_color = ['#ff4900', '#fe9a01', '#eae000', '#268b4a', '1e88f7', '#2d1f5d', '#8d0ae0' ]
lat_mean = df_result['y'].astype('float').mean()
long_mean = df_result['x'].astype('float').mean()
m = folium.Map([lat_mean, long_mean], zoom_start = 14)

folium_color = []
for idx, n in enumerate(df_result['subj_code'].unique()): # enumerate를 생각해낸게 해결법에 가까워진듯.
    folium_color.append(df_result[df_result['subj_code'] == n])    
    for k in folium_color[idx].index:
        tooltip = df_result.loc[k, 'codename'] + ' - ' + df_result.loc[k, 'fac_name'] + ' - ' + df_result.loc[k, 'addr']
        lat = df_result.loc[k, 'y']
        long = df_result.loc[k, 'x']
        folium.CircleMarker(
            location = [lat, long],
            radius = 3,
            color = hex_color[idx],
            tooltip = tooltip
        ).add_to(m)
m
m.save('ch15_project_portfolio.html')

관광지위치 확인을 희망하는 행정구역(구)를 입력해주세요종로구
관광지위치 확인을 희망하는 행정구역(구)를 입력해주세요중구
관광지위치 확인을 희망하는 행정구역(구)를 입력해주세요서대문구
관광지위치 확인을 희망하는 행정구역(구)를 입력해주세요


In [120]:
df_result.columns

Index(['fac_code', 'subj_code', 'codename', 'fac_name', 'addr', 'level2', 'x',
       'y'],
      dtype='object')

In [101]:
df_list[0]

,fac_code,subj_code,codename,fac_name,addr,level2,x,y
8,101015,1,공연장,허리우드 스카이쇼 (낙원상가),서울특별시 종로구 삼일대로 428,종로구,126.988052633,37.572747811
62,101211,1,공연장,아르코예술극장,서울특별시 종로구 대학로8길 7,종로구,127.002973678,37.581205952
70,101190,1,공연장,아이들극장,서울특별시 종로구 성균관로 91,종로구,126.998780283,37.590489023
92,100931,1,공연장,JCC 아트센터,서울 종로구 혜화동 창경궁로35길 29,종로구,127.001933427,37.587231130
102,100926,1,공연장,상명아트센터,서울 종로구 홍지동 7-1,종로구,126.957023905,37.602565951
...,...,...,...,...,...,...,...,...
792,101406,11,기타,유담헌,서울특별시 종로구 북촌로 76,종로구,126.985308696,37.583598671
794,101435,11,기타,조계사,서울특별시 종로구 우정국로 55,종로구,126.982369114,37.573694314
893,101144,11,기타,북촌문화센터,서울특별시 종로구 계동길 37,종로구,126.986428965,37.579091850
894,101141,11,기타,창신소통공작소,서울특별시 종로구 창신6가길 47,종로구,127.012400133,37.576218962


In [127]:
df_combine['codename'].unique()

array(['공연장', '박물관/기념관', '미술관', '문화예술회관', '문화원', '도서관', '기타'],
      dtype=object)

In [95]:
dist_list

['종로구', '서대문구', '중구']

In [116]:
df_result['x']
# df_result.to_csv('ch15_temp.csv')
# print(temp.iloc[:,0].size)

8      126.988052633
62     127.002973678
70     126.998780283
92     127.001933427
102    126.957023905
           ...      
317    126.948442001
324    126.945859406
343    126.960538248
482    126.944840825
516    126.913365800
Name: x, Length: 136, dtype: object

In [80]:
print(df_addr['x'].size)
print(df_attr['addr'].size)
print(df_attr.iloc[:,0].size)

913
913
913


In [49]:
resp.content

b'{"response" : {"service" : {"name" : "address", "version" : "2.0", "operation" : "getCoord", "time" : "26(ms)"}, "status" : "NOT_FOUND", "record" : {"total" : "0", "current" : "0"}, "page" : {"total" : "1", "current" : "1", "size" : "10"}}}'

In [25]:
a

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getCoord',
   'time': '35(ms)'},
  'status': 'OK',
  'input': {'type': 'parcel', 'address': '서울특별시 중구 태평로2가 17-3'},
  'refined': {'text': '서울특별시 중구 태평로2가 17-3',
   'structure': {'level0': '대한민국',
    'level1': '서울특별시',
    'level2': '중구',
    'level3': '',
    'level4L': '태평로2가',
    'level4LC': '1114011400',
    'level4A': '명동',
    'level4AC': '1114055000',
    'level5': '17-3',
    'detail': '시청광장지하쇼핑센터'}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '126.9777098499', 'y': '37.5653401802985'}}}}

In [20]:
df_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913 entries, 0 to 912
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fac_code   913 non-null    object
 1   subj_code  913 non-null    object
 2   codename   913 non-null    object
 3   fac_name   913 non-null    object
 4   addr       913 non-null    object
dtypes: object(5)
memory usage: 35.8+ KB


In [32]:
soup

<?xml version="1.0" encoding="utf-8"?>
<SearchCulturalFacilitiesAddressService>
<list_total_count>913</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<FAC_CODE>101124</FAC_CODE>
<SUBJCODE>1</SUBJCODE>
<CODENAME>공연장</CODENAME>
<FAC_NAME>신도림 오페라하우스</FAC_NAME>
<ADDR>서울시 구로구 구로동 3-39</ADDR>
</row>
<row>
<FAC_CODE>101304</FAC_CODE>
<SUBJCODE>1</SUBJCODE>
<CODENAME>공연장</CODENAME>
<FAC_NAME>신촌문화발전소</FAC_NAME>
<ADDR>서울특별시 서대문구 연세로2나길 57</ADDR>
</row>
<row>
<FAC_CODE>101429</FAC_CODE>
<SUBJCODE>1</SUBJCODE>
<CODENAME>공연장</CODENAME>
<FAC_NAME>서울광장</FAC_NAME>
<ADDR>서울특별시 중구 태평로2가 17-3</ADDR>
</row>
<row>
<FAC_CODE>101193</FAC_CODE>
<SUBJCODE>1</SUBJCODE>
<CODENAME>공연장</CODENAME>
<FAC_NAME>오류아트홀</FAC_NAME>
<ADDR>서울특별시 구로구 경인로20가길 38, 오류문화센터 2층(오류아트홀)</ADDR>
</row>
<row>
<FAC_CODE>101183</FAC_CODE>
<SUBJCODE>1</SUBJCODE>
<CODENAME>공연장</CODENAME>
<FAC_NAME>정효아트센터</FAC_NAME>
<ADDR>서울특별시 서초구 사임당로18길 52-2</ADDR>
</row>
<row>
<FAC_CODE>101156</FAC_CODE>
<S

In [59]:
from bs4 import BeautifulSoup
import requests
import urllib.request as req
import pandas as pd

key = '7549474863796a68383358725a7975' 
url1 = 'http://openAPI.seoul.go.kr:8088/' + key + '/xml/SearchCulturalFacilitiesAddressService/1/1000'
save_attrfile_path = 'C:/Users/Performance/Desktop/Fist/src/ch15_attr_xmlfile.xml'
req.urlretrieve(url1, save_attrfile_path)

attr_xml = open(save_attrfile_path, 'r', encoding='utf-8').read() 
soup = BeautifulSoup(attr_xml, 'xml')

attr_list = []
for row in soup.select('row'):
    fac_code = row.select_one('FAC_CODE').text
    subjcode = row.select_one('SUBJCODE').text
    codename = row.select_one('CODENAME').text
    fac_name = row.select_one('FAC_NAME').text
    main_img = row.select_one('MAIN_IMG') ######AttributeError: 'NoneType' object has no attribute 'text'##########
    addr = row.select_one('ADDR').text.replace(' )','')
    attr_list.append( {'fac_code' : fac_code, 'subj_code' : subjcode, 'codename' : codename, 'fac_name' : fac_name, 'main_img' : main_img, 'addr' : addr} )
df_attr = pd.DataFrame(attr_list)

import csv
df_attr.to_csv('ch15_df_attr_final.csv', encoding='cp949')

url2 = 'http://api.vworld.kr/req/address?service=address&request=getCoord&version=2.0&crs=epsg:4326&format=json&key=EA773332-297D-38CA-9A54-76F66D21A455&type=parcel'
addr_list = []
for idx, addr in enumerate(df_attr['addr']):
    if idx == 2: ########## 정상작동 확인되면 지울 것.
        resp = requests.get(url2 + '&address=' + addr)
        addr_json = resp.json()
        if addr_json['response']['status'] == 'OK':
            dic = {}
            dic['level2'] = addr_json['response']['refined']['structure']['level2']
            dic['x'] = addr_json['response']['result']['point']['x']
            dic['y'] = addr_json['response']['result']['point']['y']
            addr_list.append(dic)

In [58]:
addr_json

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getCoord',
   'time': '24(ms)'},
  'status': 'OK',
  'input': {'type': 'parcel', 'address': '서울특별시 중구 태평로2가 17-3'},
  'refined': {'text': '서울특별시 중구 태평로2가 17-3',
   'structure': {'level0': '대한민국',
    'level1': '서울특별시',
    'level2': '중구',
    'level3': '',
    'level4L': '태평로2가',
    'level4LC': '1114011400',
    'level4A': '명동',
    'level4AC': '1114055000',
    'level5': '17-3',
    'detail': '시청광장지하쇼핑센터'}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '126.9777098499', 'y': '37.5653401802985'}}}}

In [60]:
addr_list

[{'level2': '중구', 'x': '126.9777098499', 'y': '37.5653401802985'}]

In [ ]:
# true / false 반환 확인
gu = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
      '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
      '종로구', '중구', '중랑구']
dist = input('희망하는 지역의 행정구역(구)를 입력해주세요')
dist in gu

In [6]:
# 올바른 입력값이 아닌 경우 '입력값을 다시 확인해주세요' 문장이 반환
            # -> 프로그램 전체와 함께 돌려보니 그냥 다음으로 지나가서 이후 스크립트가 진행됨.
            # 진행을 멈추고 재입력을 강요할 수 있어야 함.
dist_list = []
while True:
    dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)')
    if dist_input == '':
        break
    if (dist_input in gu) == False:
        print('입력값을 다시 확인해 주세요')
        break            
    dist_list.append(dist_input)

희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)ㄴ
입력값을 다시 확인해 주세요


In [ ]:
dist_list

In [13]:
class RaisePass(Exception):
    pass # pass인 경우 진행이 멈추질 않음...

gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
  '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
  '종로구', '중구', '중랑구' ]
dist_input = input('희망하는 지역의 행정구역명(ㅇㅇ구)을 입력해주세요')
try:
    if (dist_input in gu) == True:
        pass
except RaisePass:
    print('입력값을 다시 확인해주세요')

희망하는 지역의 행정구역(구)를 입력해주세요sasas


In [11]:
# ch04_제어문에서 loop를 벗어나는 방법의 raise를 이용하는 방법을 시도해 봄.
class RaisePass(Exception):
    pass # pass인 경우 진행이 멈추질 않음...

gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
  '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
  '종로구', '중구', '중랑구' ]
dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요')
try:
    if (dist_input in gu) == False:
        raise RaisePass
except RaisePass:
    print('입력값을 다시 확인해주세요')

희망하는 지역의 행정구역(구)를 입력해주세요asas
입력값을 다시 확인해주세요


In [19]:
class RaisePass(Exception):
    pass
gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
  '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
  '종로구', '중구', '중랑구' ]
dist_input = input('희망하는 지역의 행정구역명(ㅇㅇ구)을 입력해주세요')
try:
    if (dist_input in gu) == True:
        dist_list.append(dist_input)
    else:
        raise RaisePass
except:
    print('에러')
    
# class를 정의하는 이유가 없음...;;;
# 재입력을 요구하지도, 진행이 멈추지도 않음.

희망하는 지역의 행정구역명(ㅇㅇ구)을 입력해주세요asasa
에러


In [5]:
dist_list = []
while True:
    dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)')
    if dist_input == '':
        break
    if (dist_input in gu) == False:
        print('입력값을 다시 확인해 주세요')
        break            
    dist_list.append(dist_input)
print('진행중')

# 재입력을 요구하지도, 진행이 멈추지도 않음.

희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)ㄴ
입력값을 다시 확인해 주세요
진행중


In [10]:
# 작동은 함.
gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
  '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
  '종로구', '중구', '중랑구' ]

dist_list = []
dist_input = False
while (dist_input in gu) == False:
    dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요')
    print('정확한 행정구역 명을 입력해 주세요')
    dist_list.append(dist_input)

희망하는 지역의 행정구역(구)를 입력해주세요강서
정확한 행정구역 명을 입력해 주세요
희망하는 지역의 행정구역(구)를 입력해주세요강서구
정확한 행정구역 명을 입력해 주세요


In [16]:
# ch4_제어문 에서 flag 주는 법 응용
gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
  '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
  '종로구', '중구', '중랑구' ]

dist_list = []
while True :
    dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요')
    if (dist_input in gu) == True:
        dist_list.append(dist_input)
        break
    else:
        print('정확한 행정구역 명을 입력해 주세요')
    

희망하는 지역의 행정구역(구)를 입력해주세요걍
정확한 행정구역 명을 입력해 주세요
희망하는 지역의 행정구역(구)를 입력해주세요강서
정확한 행정구역 명을 입력해 주세요
희망하는 지역의 행정구역(구)를 입력해주세요강서구


In [ ]:
#########################################################

In [ ]:
# fn1에서의 입력도 다시 정리

In [17]:
gu = ['강남구', '강동구','강북구','강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구',\
      '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구','영등포구', '용산구', '은평구',\
      '종로구', '중구', '중랑구' ]
dist_list = []
while True:
    dist_input = input('희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)')
    if dist_input == '':
        break
    if (dist_input in gu) == True:
        dist_list.append(dist_input)
#         break # break가 있으면 올바른 입력값 하나에 입력세션이 종료됨. ''을 입력했을 때만 입력세션이 종료되도록 
    else:
        print('입력값을 다시 확인해 주세요')         

희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)ㅁㄴㅁㄴ
입력값을 다시 확인해 주세요
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)강서
입력값을 다시 확인해 주세요
희망하는 지역의 행정구역(구)를 입력해주세요 (마지막에 공백을 입력함으로써 입력을 완료함)강서구


In [ ]:
# https://datacook.tistory.com/11 지번주소, 도로명주소의 좌표변환 추천
#     -> https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do

# 사용한 데이터 제공 api
# http://data.seoul.go.kr/dataList/OA-148/A/1/datasetView.do